In [26]:
import m2
import numpy as np
import pickle
from Agent import MaxAgent
import math



In [5]:

f = open('gameplay.pckl', 'rb')
data = pickle.load(f)[1:]
stateActions = []
for start, actions in data:
    for action in actions:
        stateActions.append((start, action))
        start = start.getSuccessor(action)[0]
stateActions
    

[(Cash: 0, (1, 5, 'move')),
 (Cash: 0, (3, 2, 'move')),
 (Cash: 0, (0, 0, 'move')),
 (Cash: 26, (3, 9, 'move')),
 (Cash: 26, (0, 9, 'move')),
 (Cash: 47, (40, 45, 'tank')),
 (Cash: 7, (11, 9, 'move')),
 (Cash: 7, (16, 7, 'move')),
 (Cash: 7, (13, 4, 'move')),
 (Cash: 7, (8, 3, 'move')),
 (Cash: 7, (10, 1, 'move')),
 (Cash: 7, (0, 0, 'move')),
 (Cash: 229, (None, None, 'exit')),
 (Cash: 0, (3, 2, 'move')),
 (Cash: 0, (1, 5, 'move')),
 (Cash: 0, (0, 9, 'move')),
 (Cash: 26, (3, 9, 'move')),
 (Cash: 26, (0, 9, 'move')),
 (Cash: 47, (40, 45, 'tank')),
 (Cash: 7, (7, 6, 'move')),
 (Cash: 7, (8, 3, 'move')),
 (Cash: 7, (10, 1, 'move')),
 (Cash: 7, (0, 0, 'move')),
 (Cash: 83, (13, 4, 'move')),
 (Cash: 83, (0, 0, 'move')),
 (Cash: 135, (None, None, 'exit')),
 (Cash: 0, (4, 1, 'move')),
 (Cash: 0, (3, 5, 'move')),
 (Cash: 0, (1, 5, 'move')),
 (Cash: 0, (3, 9, 'move')),
 (Cash: 0, (0, 9, 'move')),
 (Cash: 57, (40, 45, 'tank')),
 (Cash: 17, (11, 9, 'move')),
 (Cash: 17, (19, 7, 'move')),
 (Cash:

In [6]:
def generateKdata(k):
    data = []
    agent = MaxAgent(depth = k - 1)
    for state, action in stateActions:
        index = None
        values = []
        for i, move in enumerate(state.getLegalActions()):
            if action == move:
                index = i
            nextState, reward = state.getSuccessor(move)
            reward += agent.value(nextState)[0]
            values.append(reward)
        data.append((index, values))
    return data   
                
        
        

In [44]:
def generateQdata(iters):
    data = []
    for state, action in stateActions:
        index = None
        values = []
        for i, move in enumerate(state.getLegalActions()):
            if action == move:
                index = i
            nextState, reward = state.getSuccessor(move)
            _, _, scores, _ = m2.getRollout(nextState, iters)
            values.append(scores[-1])
        data.append((index, values))
    return data   

In [105]:
qdata = generateQdata(100000)

In [87]:
kdata = generateKdata(3)

In [103]:
def calculate(data, alpha):
    arr = []
    for a, b in data:
        total = 0
        for num in b:
            total += pow(math.e, alpha * num)
        arr.append(pow(math.e, alpha * b[a]) / total)
    return np.array(arr)

In [104]:
karr = calculate(kdata, 0.1)

In [110]:
qarr = calculate(qdata, 0.04)

In [158]:
mat = np.column_stack((karr, qarr))

In [174]:
import numpy as np
from scipy.optimize import minimize

m = mat
b = np.array([1 for _ in range(51)])


def loss(x):
    return np.sum(np.square((np.dot(mat, x) - b)))

cons = ({'type': 'eq',
         'fun' : lambda x: np.sum(x) - 1.0})

res = minimize(loss, x0 =[0,0], method='SLSQP', constraints=cons, options={'disp': True})

print(res.x)



Optimization terminated successfully.    (Exit mode 0)
            Current function value: 18.71589702287629
            Iterations: 3
            Function evaluations: 13
            Gradient evaluations: 3
[0.33828651 0.66171349]


In [181]:
def loss(x):
    return -np.sum(np.log(np.dot(mat, x)))

cons = ({'type': 'eq',
         'fun' : lambda x: np.sum(x) - 1.0},
       {'type': 'ineq',
         'fun' : lambda x: x[0]},
        {'type': 'ineq',
         'fun' : lambda x: x[1]}
       )

res = minimize(loss, x0 =[0.1,0.9], method='SLSQP', constraints=cons, options={'disp': True})
print(res.x)


# argmax_model P(model | F1 F2 F3 F4 F5 ...)
#  = argmax_model P(model F1 F2 F3 ...)
#  = argmax_model P(F1 F2 F3 ... | model)P(model)
#  = argmax_model P(F1 | model) * .... P(model)
    
#  ~argmax_model P(F1 F2 F3 ... | model)



Optimization terminated successfully.    (Exit mode 0)
            Current function value: 51.24533093196085
            Iterations: 4
            Function evaluations: 17
            Gradient evaluations: 4
[0.32878234 0.67121766]


In [169]:
qdata

[(1, [0, 208, 229, 229, 0, 0, 0, 0, 0, 0, 0, 235]),
 (0, [229, 229, 0, 0, 0, 0, 0, 0, 0, 208, 255]),
 (8, [0, 0, 0, 0, 0, 0, 0, 0, 229, 0]),
 (2, [26, 235, 229, 0, 0, 0, 0, 0, 0, 0, 208]),
 (8, [0, 0, 0, 0, 0, 0, 0, 0, 229]),
 (1, [47, 229, 144, 103, 0, 0, 0, 0, 0, 0, 229]),
 (4, [7, 226, 200, 135, 229, 179, 188, 0, 229]),
 (4, [176, 150, 155, 205, 229, 0, 77, 124]),
 (3, [174, 177, 177, 229, 0, 127, 127]),
 (1, [205, 229, 229, 0, 179, 179]),
 (1, [226, 229, 0, 200, 200]),
 (2, [0, 0, 229, 0]),
 (0, [229, 169, 229]),
 (2, [0, 226, 208, 100, 0, 0, 0, 0, 0, 0, 0, 244]),
 (0, [208, 302, 0, 0, 0, 0, 0, 0, 0, 229, 244]),
 (9, [234, 0, 0, 0, 0, 0, 0, 0, 208, 313]),
 (2, [26, 193, 313, 313, 0, 0, 0, 0, 0, 0, 144]),
 (8, [313, 0, 0, 0, 0, 0, 0, 229, 234]),
 (1, [47, 234, 214, 234, 0, 0, 0, 0, 0, 0, 229]),
 (1, [7, 234, 226, 155, 234, 205, 205, 0, 229]),
 (0, [226, 176, 176, 205, 205, 0, 234, 234]),
 (0, [176, 124, 226, 174, 0, 176, 176]),
 (4, [124, 176, 0, 0, 176, 124]),
 (3, [83, 116, 124, 1

In [171]:
calculate(qdata, 0.1)

array([3.10442623e-02, 6.41593432e-02, 9.99999999e-01, 3.39607557e-01,
       9.99999999e-01, 4.99948292e-01, 3.54706179e-01, 9.11810317e-01,
       9.85033169e-01, 4.75241386e-01, 5.40288018e-01, 1.00000000e+00,
       4.99381079e-01, 2.29106085e-02, 8.24120685e-05, 9.99601879e-01,
       4.99998453e-01, 3.70522853e-04, 3.64715131e-01, 3.15828356e-01,
       1.75147587e-01, 6.56868784e-03, 4.97256839e-01, 9.86579217e-01,
       1.60409904e-02, 9.97526013e-01, 4.46935850e-01, 2.25161237e-05,
       1.30940568e-02, 3.64854549e-01, 9.99999986e-01, 3.46141942e-01,
       3.70917766e-01, 5.07089288e-02, 9.60834225e-01, 9.99999944e-01,
       4.99381071e-01, 2.48006020e-01, 9.25087071e-03, 3.33333333e-01,
       3.54995695e-01, 9.87204826e-01, 9.95418275e-01, 5.05957325e-01,
       6.67776265e-01, 9.52911063e-01, 9.84305985e-01, 9.34069642e-01,
       6.38235459e-01, 1.00000000e+00, 1.00000000e+00])

In [172]:
calculate(kdata, 0.1)

array([0.27029912, 0.12242421, 0.92433616, 0.30774912, 0.99462474,
       0.67594096, 0.03380091, 0.49631943, 0.88961738, 0.47524139,
       0.54028802, 1.        , 0.49938108, 0.40323891, 0.43893903,
       0.44411338, 0.46259981, 0.48115628, 0.67594096, 0.007542  ,
       0.01880426, 0.00499944, 0.01612746, 0.98877886, 0.01603957,
       0.49938108, 0.21412915, 0.09819707, 0.19735764, 0.12825604,
       0.99919234, 0.77907465, 0.36110313, 0.92883392, 0.96083409,
       0.99999994, 0.1999009 , 0.46737296, 0.06416651, 0.63051824,
       0.2290875 , 0.17577692, 0.03167243, 0.02050423, 0.04658937,
       0.44637908, 0.80472025, 0.93406964, 0.63823546, 1.        ,
       0.5       ])

In [177]:
np.dot(mat, np.array([0.33828651, 0.66171349]))

array([0.16856706, 0.16717573, 0.97377825, 0.34882451, 0.99762539,
       0.55284559, 0.19766207, 0.57774003, 0.77543375, 0.41012575,
       0.44598377, 0.9997913 , 0.48543448, 0.22717561, 0.16159371,
       0.77600279, 0.48579679, 0.18883081, 0.43107377, 0.15956096,
       0.14172343, 0.05954355, 0.29933995, 0.81870732, 0.09778984,
       0.77310903, 0.33690372, 0.04094064, 0.14554644, 0.27242616,
       0.99809152, 0.44185915, 0.29808542, 0.3917264 , 0.84145697,
       0.9978164 , 0.38382394, 0.33868179, 0.0732769 , 0.43384759,
       0.26137582, 0.5619065 , 0.52829418, 0.19940521, 0.27587492,
       0.54597573, 0.76202458, 0.69561024, 0.47360967, 0.99995434,
       0.83084153])